Modified from https://lectures.quantecon.org/jl/optgrowth.html

In [1]:
using QuantEcon, Optim, Distributions, Distributions, QuadGK, NamedTuples

## Some More on Numerical Integration

In [5]:
#Adaptive Quadrature
quadgk(x -> sin(x), 0, 1) #Returns tuple with the error bound on the integral

(0.4596976941318603, 5.551115123125783e-17)

In [7]:
dist = Normal(0.1,2)
@show dist
@show(minimum(dist),maximum(dist)) #The support
@show pdf(dist, 1.0)
mean(dist) ≈ quadgk(x -> x * pdf(dist,x) , minimum(dist),maximum(dist))[1] #Numerical integraiton example

g(x) = x
quadgk(x -> g(x) * pdf(dist,x) , minimum(dist),maximum(dist))[1]

dist = Distributions.Normal{Float64}(μ=0.1, σ=2.0)
minimum(dist) = -Inf
maximum(dist) = Inf
pdf(dist, 1.0) = 0.18026348123082397


0.09999999999999992

## Other Quadrature Rules
While adaptive quadrature is typically the most efficient way to get to "machine precision", it is less appropriate when we want less precicision.  Another approach is gaussian quadrature which returns weights, $w$  and nodes, $\vec{x}$ to calculate integral $\int_a^b f(x) dx \approx f(\vec{x}) \cdot w$.

For example, to calculate the integral
$$
\int_{-\infty}^{\infty} f(x) \phi(x) dx
$$
where $\phi(x)$ is the pdf of the Normal$(\mu,\sigma^2)$,

In [18]:
n = 10
dist = Normal(0.1, 2)
nodes, weights = qnwnorm(n,[mean(dist)],[var(dist)])

#Then to get integrals of a function,
g(x) = x

#Check the calculations of the means and 
@show mean(dist) ≈ dot(g.(nodes), weights)
@show var(dist) ≈ dot(nodes.^2, weights) - dot(nodes, weights)^2
typeof(dist)


mean(dist) ≈ dot(g.(nodes), weights) = true
var(dist) ≈ dot(nodes .^ 2, weights) - dot(nodes, weights) ^ 2 = true


true

## Calculating Expectations
This is a good example of generic programming.  We can add in expectation calculations specific to normal distributions, discrete distributions, etc.

In [ ]:
#This sort of general utility could be added to the Distributions library, for example.  This works for any continous valued

function expectation(f, dist::Distributions.Normal{T}; kws...) where {T} #Works for a normal distribution of any type
    false #TODO: Add in gaussian quadrature.
end


function expectation(f, dist::D; kws...) where {D <: ContinuousUnivariateDistribution}
    quadgk( x -> f(x) * pdf(dist,x) , minimum(dist), maximum(dist); kws...)[1] #Uses Gauss-Kronod adaptive quadrature
end


#For any distribution, we can use adaptive quadrature
function expectation(f, dist::D; kws...) where {D <: ContinuousUnivariateDistribution}
    quadgk( x-> f(x) * pdf(dist,x) , minimum(dist), maximum(dist); kws...)[1] #Uses Gauss-Kronod adaptive quadrature
end

#For a discretely valued expectation, could just do the sums.  SHould check it is bounded
function expectation(f, dist::D; kws...) where {D <: DiscreteUnivariateDistribution}
    @assert hasfinitesupport(dist) #Otherwise might need to modify to use a truncated pdf.    
    dot(pdf.(dist, support(dist)),f.(support(dist))) #Returns the product of the pmf and the function
end

expectation (generic function with 3 methods)

In [20]:
#Test convenience operator
dist = Normal(0.1, 2)

E(f) = expectation(f, dist; maxevals=50) #Convenience operator can pass in keywords to integration
g(x) = x
E(x -> x^2) - E(g)^2 #i.e. calculate the variance

0

In [70]:
#Works with another distribution now
dist = Exponential(.1)
@show dist
expectation(x -> x^2, dist)

dist = Distributions.Exponential{Float64}(θ=0.1)


0.02

In [73]:
# Test the discrete distribution
dist = Binomial(10, 0.5)
@show dist
@show support(dist)
@show pdf.(dist, support(dist))
E(f) = expectation(f, dist)
E(n -> n^2) - E(n -> n)^2 #i.e. calculate the variance

dist = Distributions.Binomial{Float64}(n=10, p=0.5)
support(dist) = 0:10
pdf.(dist, support(dist)) = [0.000976563, 0.00976563, 0.0439453, 0.117188, 0.205078, 0.246094, 0.205078, 0.117188, 0.0439453, 0.00976563, 0.000976563]


2.5

## Interpolation Examples

In [63]:
# Linear Interpolation
x = 0:0.1:1
@show x
y_grid = x.^2
@show y
quadtest = LinInterp(x,y)
finergrid = 0:0.001:1
using Plots
gr()
plot(finergrid, quadtest.(finergrid))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip1802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.4745,375.869 33.0223,375.832 33.5702,375.796 34.118,375.759 34.6659,375.723 35.2137,375.687 35.7615,375.65 36.3094,375.614 36.8572,375.577 37.4051,375.541 
 37.9529,375.505 38.5007,375.468 39.0486,375.432 39.5964,375.395 40.1443,375.359 40.6921,375.323 41.2399,375.286 41.7878,375.25 42.3356,375.213 42.8835,375.177 
 43.4313,375.141 43.9791,375.104 44.527,375.068 45.0748,375.031 45.6227,374.995 46.1705,374.959 46.7183,374.922 47.2662,374.886 47.814,374.849 48.3619,374.813 
 48.9097,374.777 49.4575,374.74 50.0054,374.704 50.5532,374.667 51.1011,374.631 51.6489,374.594 52.1968,374.558 52.7446,374.522 53.2924,374.485 53.8403,374.449 
 54.3881,374.412 54.936,374.376 55.4838,374.34 56.0316,374.303 56.5795,374.267 57.1273,374.23 57.6752,374.194 58.223,374.158 58.7708,374.121 59.3187,374.085 
 59.8665,374.048 60.4144,374.012 60.9622,373.976 61.51,373.939 62.0579,373.903 62.6057,373.866 63.1536,373.83 63.7014,373.794 64.2492,373.757 64.7971,373.721 
 65.3449,373.684 65.8928,373.648 66.4406,373.612 66.9884,373.575 67.5363,373.539 68.0841,373.502 68.632,373.466 69.1798,373.429 69.7276,373.393 70.2755,373.357 
 70.8233,373.32 71.3712,373.284 71.919,373.247 72.4668,373.211 73.0147,373.175 73.5625,373.138 74.1104,373.102 74.6582,373.065 75.2061,373.029 75.7539,372.993 
 76.3017,372.956 76.8496,372.92 77.3974,372.883 77.9453,372.847 78.4931,372.811 79.0409,372.774 79.5888,372.738 80.1366,372.701 80.6845,372.665 81.2323,372.629 
 81.7801,372.592 82.328,372.556 82.8758,372.519 83.4237,372.483 83.9715,372.447 84.5193,372.41 85.0672,372.374 85.615,372.337 86.1629,372.301 86.7107,372.265 
 87.2585,372.228 87.8064,372.119 88.3542,372.01 88.9021,371.9 89.4499,371.791 89.9977,371.682 90.5456,371.573 91.0934,371.464 91.6413,371.354 92.1891,371.245 
 92.7369,371.136 93.2848,371.027 93.8326,370.917 94.3805,370.808 94.9283,370.699 95.4761,370.59 96.024,370.481 96.5718,370.371 97.1197,370.262 97.6675,370.153 
 98.2153,370.044 98.7632,369.935 99.311,369.825 99.8589,369.716 100.407,369.607 100.955,369.498 101.502,369.388 102.05,369.279 102.598,369.17 103.146,369.061 
 103.694,368.952 104.242,368.842 104.789,368.733 105.337,368.624 105.885,368.515 106.433,368.405 106.981,368.296 107.529,368.187 108.076,368.078 108.624,367.969 
 109.172,367.859 109.72,367.75 110.268,367.641 110.816,367.532 111.364,367.423 111.911,367.313 112.459,367.204 113.007,367.095 113.555,366.986 114.103,366.876 
 114.651,366.767 115.198,366.658 115.746,366.549 116.294,366.44 116.842,366.33 117.39,366.221 117.938,366.112 118.485,366.003 119.033,365.894 119.581,365.784 
 120.129,365.675 120.677,365.566 121.225,365.457 121.772,365.347 122.32,365.238 122.868,365.129 123.416,365.02 123.964,364.911 124.512,364.801 125.06,364.692 
 125.607,364.583 126.155,364.474 126.703,364.364 127.251,364.255 127.799,364.146 128.347,364.037 128.894,363.928 129.442,363.818 129.99,363.709 130.538,363.6 
 131.086,363.491 131.634,363.382 132.181,363.272 132.729,363.163 133.277,363.054 133.825,362.945 134.373,362.835 134.921,362.726 135.469,362.617 136.016,362.508 
 136.564,362.399 137.112,362.289 137.66,362.18 138.208,362.071 138.756,361.962 139.303,361.852 139.851,361.743 140.399,361.634 140.947,361.525 141.495,361.416 
 142.043,361.306 142.59,361.124 143.138,360.942 143.686,360.76 144.234,360.578 144.782,360.396 145.33,360.214 145.877,360.032 146.425,359.85 146.973,359.668 
 147.521,359.486 148.069,359.304 148.617,359.122 149.165,358.94 149.712,358.758 150.26,358.576 150.808,358.394 151.356,358.212 151.904,358.03 152.452,357.848 
 152.999,357.666 153.547,357.484 154.095,357.302 154.643,357.12 155.191,356.938 155.739,356.756 156.286,356.574 156.834,356

## Solving the Closed Form Problem

In [78]:
α = 0.4
β = 0.96
μ = 0
s = 0.1

c1 = log(1 - α * β) / (1 - β)
c2 = (μ + α * log(α * β)) / (1 - α)
c3 = 1 / (1 - β)
c4 = 1 / (1 - α * β)

# Utility 
u(c) = log(c)

u_prime(c) = 1 / c

# Deterministic part of production function
f(k) = k^α

f_prime(k) = α * k^(α - 1)

# True optimal policy
c_star(y) = (1 - α * β) * y

# True value function
v_star(y) = c1 + c2 * (c3 - c4) + c4 * log(y)

# Grid
grid_max = 4         # Largest grid point
grid_size = 200      # Number of grid points
grid_y = collect(linspace(1e-5, grid_max, grid_size))


200-element Array{Float64,1}:
 1.0e-5   
 0.0201105
 0.0402109
 0.0603114
 0.0804118
 0.100512 
 0.120613 
 0.140713 
 0.160814 
 0.180914 
 0.201015 
 0.221115 
 0.241215 
 ⋮        
 3.7789   
 3.799    
 3.8191   
 3.8392   
 3.8593   
 3.8794   
 3.8995   
 3.9196   
 3.9397   
 3.9598   
 3.9799   
 4.0      

In [114]:

function T(w_grid, params)
    grid, β, u, f, E = params.grid, params.β, params.u, params.f, params.E #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid

    return [-optimize(c -> - ( #Negates because this is a minimizer
                u(c) + β * E( ζ -> w(f(y - c) * exp(ζ)) ) #objective
                ), 1e-10, y).minimum for y in grid] #for the whole grid
end

T (generic function with 1 method)

In [131]:
#Plotting the results
using Plots
gr()
plot(grid_y, w, lw=2, alpha=0.6, label="Tv^*")
E(f) = expectation(f, Normal(μ,s^2); maxevals=200) #convenience operator for expectations

params = @NT(grid=grid_y, β = β, u = log, f = f, E = E)
w = T(v_star.(grid_y), params) #Shock distribution
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="v^")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip4602)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,128.92 41.6256,113.602 44.3601,104.614 47.0946,98.2586 49.829,93.3211 52.5635,89.2859 55.298,85.8745 58.0324,82.9181 60.7669,80.3094 
 63.5014,77.9786 66.2358,75.8724 68.9703,73.945 71.7047,72.174 74.4392,70.5359 77.1737,69.0074 79.9081,67.5814 82.6426,66.2383 85.3771,64.9751 88.1115,63.7776 
 90.846,62.644 93.5805,61.5633 96.3149,60.5352 99.0494,59.5507 101.784,58.6098 104.518,57.7062 107.253,56.8384 109.987,56.0039 112.722,55.1986 115.456,54.4228 
 118.191,53.6726 120.925,52.9468 123.66,52.2449 126.394,51.5638 129.128,50.9032 131.863,50.2622 134.597,49.6387 137.332,49.0324 140.066,48.4427 142.801,47.8678 
 145.535,47.3075 148.27,46.7616 151.004,46.2283 153.739,45.7075 156.473,45.199 159.208,44.702 161.942,44.2155 164.677,43.7397 167.411,43.2741 170.145,42.8178 
 172.88,42.3707 175.614,41.9327 178.349,41.5032 181.083,41.0817 183.818,40.6681 186.552,40.2622 189.287,39.8636 192.021,39.472 194.756,39.0871 197.49,38.7089 
 200.225,38.3372 202.959,37.9715 205.693,37.6117 208.428,37.2577 211.162,36.9094 213.897,36.5663 216.631,36.2285 219.366,35.8958 222.1,35.5681 224.835,35.245 
 227.569,34.9267 230.304,34.6129 233.038,34.3034 235.773,33.9984 238.507,33.6973 241.241,33.4003 243.976,33.1073 246.71,32.8181 249.445,32.5326 252.179,32.2508 
 254.914,31.9725 257.648,31.6977 260.383,31.4262 263.117,31.158 265.852,30.8931 268.586,30.6312 271.321,30.3725 274.055,30.1167 276.79,29.8638 279.524,29.6139 
 282.258,29.3667 284.993,29.1222 287.727,28.8804 290.462,28.6413 293.196,28.4046 295.931,28.1705 298.665,27.9388 301.4,27.7096 304.134,27.4827 306.869,27.2581 
 309.603,27.0357 312.338,26.8156 315.072,26.5976 317.806,26.3817 320.541,26.168 323.275,25.9563 326.01,25.7466 328.744,25.5389 331.479,25.333 334.213,25.1291 
 336.948,24.9271 339.682,24.7269 342.417,24.5285 345.151,24.3318 347.886,24.1369 350.62,23.9437 353.354,23.7521 356.089,23.5622 358.823,23.3739 361.558,23.1872 
 364.292,23.0021 367.027,22.8185 369.761,22.6364 372.496,22.4558 375.23,22.2767 377.965,22.099 380.699,21.9226 383.434,21.7478 386.168,21.5743 388.903,21.4021 
 391.637,21.2312 394.371,21.0617 397.106,20.8935 399.84,20.7265 402.575,20.5608 405.309,20.3963 408.044,20.233 410.778,20.0709 413.513,19.91 416.247,19.7503 
 418.982,19.5917 421.716,19.4342 424.451,19.2779 427.185,19.1227 429.919,18.9685 432.654,18.8154 435.388,18.6633 438.123,18.5123 440.857,18.3623 443.592,18.2133 
 446.326,18.0654 449.061,17.9183 451.795,17.7723 454.53,17.6272 457.264,17.4832 459.999,17.3399 462.733,17.1976 465.467,17.0563 468.202,16.9158 470.936,16.7762 
 473.671,16.6375 476.405,16.4997 479.14,16.3627 481.874,16.2265 484.609,16.0913 487.343,15.9567 490.078,15.8231 492.812,15.6902 495.547,15.5581 498.281,15.4269 
 501.016,15.2962 503.75,15.1665 506.484,15.0376 509.219,14.9093 511.953,14.7818 514.688,14.655 517.422,14.529 520.157,14.4036 522.891,14.2789 525.626,14.155 
 528.36,14.0318 531.095,13.9092 533.829,13.7873 536.564,13.6661 539.298,13.5455 542.032,13.4256 544.767,13.3063 547.501,13.1877 550.236,13.0697 552.97,12.9523 
 555.705,12.8356 558.439,12.7195 561.174,12.6039 563.908,12.489 566.643,12.3746 569.377,12.2609 572.112,12.1478 574.846,12.0352 577.58,11.9231 580.315,11.8117 
 
 "/>
<polyline clip-path="url(#clip4602)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,297.182 38.8912,128.905 41.6256,113.575 44.3601,104.607 47.0946,98.2436 49.829,93.3075 52.5635,89.2744 55.298,85.8643 58.0324,82.9104 60.7669,80.3048 
 63.5014,77.9741 66.2358,75.8656 68.9703,73.9407 71.7047,72.17 74.4392,70.5305 77.1737,69.0043 79.9081,67.5765 82.6426,66.2354 85.3771,64.9709 88.1115,63.7748 
 90.846,62.64 93.5805,61.5607 96

In [132]:
E(f) = expectation(f, Normal(μ,s^2); maxevals=50)
params = @NT(grid=grid_y, β = β, u = log, f = f, E = E)
w = 5 * log.(grid_y)  # An initial condition -- fairly arbitrary
n = 50

lb = "initial condition"
p = plot(grid_y, w)

for i = 1:n
    w = T(w, params)
    plot!(grid_y, w)    
end
   
p
lb = "true value function"
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.8, label=lb)

#show()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
<polyline clip-path="url(#clip4802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,375.869 38.8912,161.191 41.6256,141.635 44.3601,130.194 47.0946,122.076 49.829,115.779 52.5635,110.634 55.298,106.283 58.0324,102.515 60.7669,99.1909 
 63.5014,96.2175 66.2358,93.5276 68.9703,91.072 71.7047,88.813 74.4392,86.7215 77.1737,84.7744 79.9081,82.953 82.6426,81.242 85.3771,79.6288 88.1115,78.1029 
 90.846,76.6553 93.5805,75.2783 96.3149,73.9654 99.0494,72.7109 101.784,71.5097 104.518,70.3576 107.253,69.2507 109.987,68.1856 112.722,67.1592 115.456,66.1688 
 118.191,65.212 120.925,64.2866 123.66,63.3906 126.394,62.5221 129.128,61.6796 131.863,60.8614 134.597,60.0664 137.332,59.2931 140.066,58.5405 142.801,57.8074 
 145.535,57.0928 148.27,56.3959 151.004,55.7158 153.739,55.0517 156.473,54.4029 159.208,53.7686 161.942,53.1483 164.677,52.5414 167.411,51.9472 170.145,51.3652 
 172.88,50.795 175.614,50.2362 178.349,49.6881 181.083,49.1505 183.818,48.623 186.552,48.1051 189.287,47.5966 192.021,47.097 194.756,46.6062 197.49,46.1237 
 200.225,45.6494 202.959,45.1829 205.693,44.724 208.428,44.2724 211.162,43.8279 213.897,43.3903 216.631,42.9595 219.366,42.535 222.1,42.1169 224.835,41.7049 
 227.569,41.2988 230.304,40.8985 233.038,40.5037 235.773,40.1144 238.507,39.7304 241.241,39.3516 243.976,38.9778 246.71,38.6089 249.445,38.2447 252.179,37.8851 
 254.914,37.5301 257.648,37.1795 260.383,36.8332 263.117,36.4911 265.852,36.1531 268.586,35.8191 271.321,35.489 274.055,35.1627 276.79,34.8402 279.524,34.5213 
 282.258,34.2059 284.993,33.8941 287.727,33.5856 290.462,33.2805 293.196,32.9787 295.931,32.68 298.665,32.3845 301.4,32.092 304.134,31.8025 306.869,31.516 
 309.603,31.2323 312.338,30.9515 315.072,30.6734 317.806,30.3981 320.541,30.1254 323.275,29.8553 326.01,29.5878 328.744,29.3228 331.479,29.0603 334.213,28.8001 
 336.948,28.5424 339.682,28.287 342.417,28.0338 345.151,27.783 347.886,27.5343 350.62,27.2878 353.354,27.0435 356.089,26.8012 358.823,26.561 361.558,26.3228 
 364.292,26.0867 367.027,25.8524 369.761,25.6201 372.496,25.3898 375.23,25.1612 377.965,24.9345 380.699,24.7096 383.434,24.4865 386.168,24.2652 388.903,24.0455 
 391.637,23.8276 394.371,23.6113 397.106,23.3967 399.84,23.1837 402.575,22.9723 405.309,22.7625 408.044,22.5542 410.778,22.3474 413.513,22.1421 416.247,21.9384 
 418.982,21.736 421.716,21.5352 424.451,21.3357 427.185,21.1377 429.919,20.941 432.654,20.7457 435.388,20.5517 438.123,20.359 440.857,20.1677 443.592,19.9776 
 446.326,19.7888 449.061,19.6013 451.795,19.415 454.53,19.23 457.264,19.0461 459.999,18.8634 462.733,18.6819 465.467,18.5016 468.202,18.3224 470.936,18.1443 
 473.671,17.9674 476.405,17.7915 479.14,17.6168 481.874,17.4431 484.609,17.2705 487.343,17.0989 490.078,16.9284 492.812,16.7589 495.547,16.5904 498.281,16.4229 
 501.016,16.2564 503.75,16.0908 506.484,15.9263 509.219,15.7626 511.953,15.6 514.688,15.4382 517.422,15.2774 520.157,15.1175 522.891,14.9585 525.626,14.8004 
 528.36,14.6432 531.095,14.4868 533.829,14.3313 536.564,14.1767 539.298,14.0228 542.032,13.8699 544.767,13.7177 547.501,13.5664 550.236,13.4159 552.97,13.2661 
 555.705,13.1172 558.439,12.9691 561.174,12.8217 563.908,12.6751 566.643,12.5292 569.377,12.3841 572.112,12.2397 574.846,12.0961 577.58,11.9532 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip4802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,337.037 38.8912,124.762 41.6256,113.53 44.3601,106.955 47.0946,102.288 49.829,98.6731 52.5635,95.7143 55.298,93.2174 58.0324,91.054 60.7669,89.1454 
 63.5014,87.4376 66.2358,85.8931 68.9703,84.485 71.7047,83.1884 74.4392,81.9864 77.1737,80.8679 79.9081,79.8237 82.6426,78.8404 85.3771,77.9137 88.1115,77.0389 
 90.846,76.2069 93.5805,75.417 96.3149,74.6629 9

In [137]:
function solve_optgrowth(initial_w, params,
                         tol::AbstractFloat=1e-6,
                         max_iter::Integer=500)

    w = initial_w  # Set initial condition
    error = tol + 1
    i = 0

    # == Create storage array for bellman_operator. Reduces  memory
    # allocation and speeds code up == #
    Tw = similar(grid_y)

    # Iterate to find solution

    while (error > tol) && (i < max_iter)
        w_new = T(w,params)
        error = maximum(abs, w_new - w)
        w = w_new
        i += 1
    end

    return w
end

solve_optgrowth (generic function with 3 methods)

In [138]:
initial_w = 5 * log.(grid_y)
v_star_approx = solve_optgrowth(initial_w, params)

plot(grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
 
 -20 
 
 
 -15 
 
<polyline clip-path="url(#clip5002)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,107.193 41.6256,94.7129 44.3601,87.399 47.0946,82.2169 49.829,78.2 52.5635,74.9128 55.298,72.133 58.0324,69.7244 60.7669,67.6006 
 63.5014,65.7055 66.2358,63.9873 68.9703,62.4173 71.7047,60.9774 74.4392,59.6403 77.1737,58.3972 79.9081,57.2339 82.6426,56.1407 85.3771,55.1116 88.1115,54.137 
 90.846,53.213 93.5805,52.3334 96.3149,51.4949 99.0494,50.6947 101.784,49.9267 104.518,49.1916 107.253,48.483 109.987,47.8032 112.722,47.1485 115.456,46.5155 
 118.191,45.9053 120.925,45.3144 123.66,44.7422 126.394,44.1881 129.128,43.6492 131.863,43.1271 134.597,42.62 137.332,42.1259 140.066,41.6456 142.801,41.1778 
 145.535,40.7214 148.27,40.2763 151.004,39.8423 153.739,39.4184 156.473,39.0038 159.208,38.5992 161.942,38.2033 164.677,37.8153 167.411,37.436 170.145,37.0646 
 172.88,36.7004 175.614,36.343 178.349,35.9933 181.083,35.65 183.818,35.3134 186.552,34.9825 189.287,34.6582 192.021,34.3393 194.756,34.026 197.49,33.7178 
 200.225,33.4152 202.959,33.1173 205.693,32.8243 208.428,32.5356 211.162,32.252 213.897,31.9728 216.631,31.6977 219.366,31.4264 222.1,31.1597 224.835,30.897 
 227.569,30.6376 230.304,30.382 233.038,30.1299 235.773,29.8815 238.507,29.6365 241.241,29.3944 243.976,29.1555 246.71,28.92 249.445,28.6877 252.179,28.4581 
 254.914,28.2313 257.648,28.0072 260.383,27.7863 263.117,27.568 265.852,27.3523 268.586,27.1389 271.321,26.928 274.055,26.7199 276.79,26.5141 279.524,26.3106 
 282.258,26.1091 284.993,25.9099 287.727,25.713 290.462,25.5183 293.196,25.3258 295.931,25.1348 298.665,24.946 301.4,24.7591 304.134,24.5745 306.869,24.3917 
 309.603,24.2107 312.338,24.0311 315.072,23.8536 317.806,23.6779 320.541,23.5039 323.275,23.3317 326.01,23.1609 328.744,22.9915 331.479,22.8237 334.213,22.6578 
 336.948,22.4934 339.682,22.3304 342.417,22.1687 345.151,22.0083 347.886,21.8493 350.62,21.6921 353.354,21.5361 356.089,21.3816 358.823,21.2282 361.558,21.0759 
 364.292,20.925 367.027,20.7756 369.761,20.6274 372.496,20.4804 375.23,20.3346 377.965,20.1896 380.699,20.0459 383.434,19.9034 386.168,19.7623 388.903,19.6222 
 391.637,19.4831 394.371,19.3449 397.106,19.2077 399.84,19.0716 402.575,18.9367 405.309,18.8028 408.044,18.6699 410.778,18.5379 413.513,18.4066 416.247,18.2762 
 418.982,18.1469 421.716,18.0189 424.451,17.8916 427.185,17.7652 429.919,17.6397 432.654,17.5148 435.388,17.3907 438.123,17.2677 440.857,17.1457 443.592,17.0245 
 446.326,16.9041 449.061,16.7844 451.795,16.6654 454.53,16.547 457.264,16.4295 459.999,16.313 462.733,16.1973 465.467,16.0822 468.202,15.9679 470.936,15.8542 
 473.671,15.741 476.405,15.6287 479.14,15.5171 481.874,15.4064 484.609,15.2963 487.343,15.1868 490.078,15.0779 492.812,14.9696 495.547,14.8618 498.281,14.7548 
 501.016,14.6486 503.75,14.5431 506.484,14.438 509.219,14.3337 511.953,14.2298 514.688,14.1265 517.422,14.0236 520.157,13.9215 522.891,13.8202 525.626,13.7194 
 528.36,13.619 531.095,13.5193 533.829,13.42 536.564,13.3212 539.298,13.2228 542.032,13.1251 544.767,13.0281 547.501,12.9316 550.236,12.8355 552.97,12.74 
 555.705,12.645 558.439,12.5503 561.174,12.4561 563.908,12.3624 566.643,12.2693 569.377,12.1768 572.112,12.0847 574.846,11.9931 577.58,11.9019 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip5002)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,368.439 38.8912,229.174 41.6256,216.488 44.3601,209.066 47.0946,203.8 49.829,199.715 52.5635,196.377 55.298,193.555 58.0324,191.11 60.7669,188.954 
 63.5014,187.025 66.2358,185.28 68.9703,183.687 71.7047,182.221 74.4392,180.865 77.1737,179.601 79.9081,178.42 82.6426,177.31 85.3771,176.263 88.1115,175.274 
 90.846,174.334 93.5805,173.441 96.3149

In [ ]:
import QuantEcon: compute_fixed_point

Tw = similar(grid_y)
initial_w = 5 * log.(grid_y)

bellman_operator(w) = bellman_operator(w,
                                       grid_y,
                                       β,
                                       log,
                                       k -> k^α,
                                       shocks)

v_star_approx = compute_fixed_point(bellman_operator,
                                    initial_w,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    err_tol=1e-5)

```none
Compute iterate 10 with error 0.709153897728406
Compute iterate 20 with error 0.47095844432889145
Compute iterate 30 with error 0.3131085083453158
Compute iterate 40 with error 0.20816475495214704
Compute iterate 50 with error 0.13839472275577336
Compute iterate 60 with error 0.09200932833702069
Compute iterate 70 with error 0.061170804294146564
Compute iterate 80 with error 0.04066834706496181
Compute iterate 90 with error 0.027037644380378367
Compute iterate 100 with error 0.01797550838462314
Compute iterate 110 with error 0.011950704623018282
Compute iterate 120 with error 0.00794521845620011
Compute iterate 130 with error 0.005282240529794535
Compute iterate 140 with error 0.003511805895449527
Compute iterate 150 with error 0.0023347631704915273
Compute iterate 160 with error 0.001552226753144481
Compute iterate 170 with error 0.0010319710041208907
Compute iterate 180 with error 0.0006860880028760619
Compute iterate 190 with error 0.0004561336956570017
Compute iterate 200 with error 0.00030325256715002524
Compute iterate 210 with error 0.00020161220411551994
Compute iterate 220 with error 0.0001340383727708172
Compute iterate 230 with error 8.911308481529545e-5
Compute iterate 240 with error 5.924528747414115e-5
Compute iterate 250 with error 3.9388200878676116e-5
Compute iterate 260 with error 2.6186561168373146e-5
Compute iterate 270 with error 1.7409680953761608e-5
Compute iterate 280 with error 1.1574523867352582e-5
Converged in 284 steps
```


In [ ]:
fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
Tw, σ = bellman_operator(v_star_approx,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks;
                         compute_policy=true)


cstar = (1 - α * β) * grid_y

fig, ax = subplots(figsize=(9, 5))
ax[:plot](grid_y, σ, lw=2, alpha=0.6, label="approximate policy function")
ax[:plot](grid_y, cstar, lw=2, alpha=0.6, label="true policy function")
ax[:legend](loc="lower right")
show()

In [ ]:
s = 0.05
shocks = exp.(μ + s * randn(shock_size))

In [ ]:
"""
Compute a time series given consumption policy σ.
"""
function simulate_og(σ, y0 = 0.1, ts_length=100)
    y = Array{Float64}(ts_length)
    ξ = randn(ts_length-1)
    y[1] = y0
    for t in 1:(ts_length-1)
        y[t+1] = (y[t] - σ(y[t]))^α * exp(μ + s * ξ[t])
    end
    return y
end

fig, ax = subplots(figsize=(9, 6))

for β in (0.9, 0.94, 0.98)

    Tw = similar(grid_y)
    initial_w = 5 * log.(grid_y)

    v_star_approx = compute_fixed_point(bellman_operator,
                                        initial_w,
                                        max_iter=50,
                                        verbose=0,
                                        print_skip=10,
                                        err_tol=1e-5)

    Tw, σ = bellman_operator(v_star_approx,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks,
                             compute_policy=true)

    σ_func = LinInterp(grid_y, σ)
    y = simulate_og(σ_func)
    ax[:plot](y, lw=2, alpha=0.6, label="β = $β" )
end


ax[:legend](loc="lower right")
show()